Docker image built with [Dockerfile](https://github.com/palmoreck/dockerfiles/blob/master/jupyterlab/kale/opt/0.1/Dockerfile)

In [ ]:
import numpy as np
import sympy
from sympy.tensor.array import derive_by_array

from opt.classes.problems import ProblemFeasibleInitPoint
from opt.classes.functions import LogarithmicBarrier, GradientLogarithmicBarrier, \
HessianLogarithmicBarrier

In [ ]:

def map_sympy_vars_to_array_values(x):
    """
    Auxiliary function to evaluate derivatives of sympy which are defined
    in terms of symbols x1, x2.
    Args:
        x (numpy ndarray): 1d array of floats
    Returns:
        dictionary of the form {x1: x[0], x2: x[1],...} to map sympy Symbols
        to values.
    """
    n = x.size
    return {"x%s" % i: x[i-1] for i in range(1,n+1)}

def gf_sympy_func(x):
    x1, x2 = sympy.symbols("x1, x2")
    x_sympy = (x1, x2)
    fo_sympy = -3*x1 -5*x2
    return derive_by_array(fo_sympy, x_sympy)

def gf(x):
    gf_sympy = gf_sympy_func(x)
    return np.array([first_partial_derivative.subs(map_sympy_vars_to_array_values(x)) \
                     for first_partial_derivative in gf_sympy],
                     dtype = float)

def Hf(x):
    x1, x2 = sympy.symbols("x1, x2")
    x_sympy = (x1, x2)
    gf_sympy = gf_sympy_func(x)
    Hf_sympy = derive_by_array(gf_sympy, x_sympy)
    return np.array([second_partial_derivative.subs(map_sympy_vars_to_array_values(x)) \
                     for second_partial_derivative in Hf_sympy],
                     dtype=float).reshape(n,n)

def gphi_sympy_func(x):
    x1, x2 = sympy.symbols("x1, x2")
    x_sympy = (x1, x2)
    phi_sympy = -(sympy.log(4-x1) + sympy.log(12-2*x2) + sympy.log(18-3*x1-2*x2) + sympy.log(x1) + sympy.log(x2))  
    return derive_by_array(phi_sympy, x_sympy)
def gphi(x):
    gphi_sympy = gphi_sympy_func(x)
    return np.array([first_partial_derivative.subs(map_sympy_vars_to_array_values(x)) \
                     for first_partial_derivative in gphi_sympy],
                     dtype = float)

def Hphi(x):
    x1, x2 = sympy.symbols("x1, x2")
    x_sympy = (x1, x2)
    gphi_sympy = gphi_sympy_func(x)
    Hphi_sympy = derive_by_array(gphi_sympy,x_sympy)
    return np.array([second_partial_derivative.subs(map_sympy_vars_to_array_values(x)) \
                     for second_partial_derivative in Hphi_sympy],
                    dtype=float).reshape(n,n)

#definition of objective function with numpy

def fo_numpy(x):
    return np.dot(np.array([-3, -5]), x)


In [ ]:
c = np.array([-3, -5])
b = np.array([4, 12, 18], dtype=float)
x_0 = np.array([1, 2], dtype=float)
n = x_0.size

gf_class = GradientLogarithmicBarrier(gf, gphi)
Hf_class = HessianLogarithmicBarrier(Hf, Hphi)

#parameters for logarithmic barrier method
mu = 10
t_B_0 = 10

#solution
x_ast = np.array([2, 6], dtype=float)
p_ast = fo_numpy(x_ast)

#constraints inequalities definition as < = 0

constraints_ineq = {0: lambda x: x[0] - 4,
                    1: lambda x: 2*x[1] - 12,
                    2: lambda x: 3*x[0] + 2*x[1] - 18,
                    3: lambda x: -x[0],
                    4: lambda x: -x[1]
                    }

In [ ]:
type_problem = "CICO" #Constrained Inequality Convex Optimization
p = ProblemFeasibleInitPoint(type_problem, 
                             fo_numpy,
                             mu=mu,
                             constraints_ineq=constraints_ineq)

In [ ]:
if p.well_defined_problem:
    print("problema %s bien definido" % p.type_problem)
    [x,total_iter,t_B,x_plot] = p.solve(x_0,
                                        t_B_0,
                                        x_ast=x_ast,
                                        p_ast=p_ast,
                                        gf_B=gf_class,
                                        Hf_B=Hf_class
                                       )
else:
    print("problema %s no bien definido" % p.type_problem)
